# Curso: Redes Neurais e Deep Learning

Prof. Denilson Alves Pereira 
https://sites.google.com/ufla.br/denilsonpereira/ 
Departamento de Ciência da Computação - 
Instituto de Ciências Exatas e Tecnológicas - 
Universidade Federal de Lavras

# Problema

Elaborar uma rede neural artifical (ANN) para predizer se uma amostra de água é considerada potável, com base em medidas contidas no dataset disponível em https://www.kaggle.com/datasets/adityakadiwal/water-potability

Os parâmetros de entrada para o problema são:
- valor do PH
- dureza, causada por cálcio e sais de magnésio
- total de sólidos dissolvidos
- concentração de cloraminas
- concentração de sulfato
- condutividade
- concentração de carbono orgânico
- concentração de trialometanos
- turbidez

E a saída é binária de acordo com que a água seja potável ou não.

# Pacotes

In [ ]:
import numpy as np # para computação científica
import tensorflow as tf # para computação númerica nos dados
from tensorflow import keras # para deep learning
import pandas as pd # para trabalhar com análise de dados

# Pré-processamento dos dados de treino e teste

## Lendo o dataset

In [ ]:
data = pd.read_csv('potabilidade.csv') # lẽ o dataset
data.head(10) # mostra as 10 primeiras linhas do dataset

## Removendo linhas que possuem ausência de dados no dataset

A forma de lidar com a ausência de dados foi remover a linha completa onde um ou mais valores nulos (NaN) forem encontrados. A consequência disso é uma enorme redução do dataset disponível.

In [ ]:
data.dropna(axis="index", how='any', inplace=True) # remove linhas onde for encontrado valor nulo
data.head(10) # mostra as 10 primeiras linhas do dataset após a remocão

## Separando as entradas e saídas

### Entradas

In [ ]:
X = data.drop("Potability", axis=1)
X.head(10)

### Saídas

In [ ]:
Y = data["Potability"]
Y.head(10)

## Criando conjuntos de treino e teste para as entradas e saídas

In [ ]:
from sklearn.model_selection import train_test_split
train_set_X, test_set_X, train_set_Y, test_set_Y = train_test_split(X, Y, test_size=0.20)

## Padronizando os atributos

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_set_X = scaler.fit_transform(train_set_X)
test_set_X  = scaler.fit_transform(test_set_X)

## Obtendo número de atributos e exemplos de treinamento

In [ ]:
n = train_set_X.shape[1] # número de atributos
m = train_set_X.shape[0] # número de exemplos para treinamento

print ("Número de atributos: n = " + str(n))
print ("Número de exemplos para treinamento: m = " + str(m))

# Definição do modelo
- Camada de entrada: de acordo com as 9 entradas do problema
- Camada 1: 7 neurônios, função de ativação *ReLu*
- Camada 2 (saída): 1 neurônio, função de ativação *Sigmoid*

In [ ]:
inputs = keras.Input(shape=n)
x = keras.layers.Dense(units=7, activation="relu")(inputs)
outputs = keras.layers.Dense(units=1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

# Compilação do modelo

In [ ]:
model.compile(optimizer="RMSprop", loss="mean_absolute_error", metrics=["accuracy", "Precision", "Recall"])

# Treinamento do modelo

In [ ]:
model.fit(train_set_X, train_set_Y, batch_size=32, epochs=500)

# Avaliação do modelo

In [ ]:
loss, acc, prec, rec = model.evaluate(test_set_X, test_set_Y)
print("Loss: %.2f" % loss, "\nAccuracy: %.2f" % acc, "\nPrecision: %.2f" % prec, "\nRecall: %.2f" % rec)

# Predição

In [ ]:
predictions = model.predict(test_set_X)
print("Predição: ", [round(x[0]) for x in predictions])
print()
print("Correto: ", [round(x) for x in test_set_Y])